In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
PATH_OF_IMAGES = './raw-img/'
TARGET_SIZE = (128, 128)
BATCH_SIZE = 256
EPOCHS = 40
CLASSES = 10

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)  # 20% az adatoknak validációs készlet lesz


In [ ]:
train_generator = datagen.flow_from_directory(
    PATH_OF_IMAGES,  # Az összes kép tartalmazó mappa
    target_size=TARGET_SIZE,   # Célméret a képek átméretezésére
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # Vagy 'binary', ha kevesebb mint két osztályod van
    subset='training')    # Tanító adatkészlet

validation_generator = datagen.flow_from_directory(
    PATH_OF_IMAGES,  # Ugyanaz a mappa, mint a tanítónál
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # Vagy 'binary'
    subset='validation')  # Validációs adatkészlet


In [ ]:
model = Sequential([
    # Első konvolúciós réteg
    Conv2D(32, (3, 3), activation='relu', input_shape=(TARGET_SIZE[0], TARGET_SIZE[1], 3)),
    MaxPooling2D(2, 2),
    
    # Második konvolúciós réteg
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    # Harmadik konvolúciós réteg
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    # Negyedik konvolúciós réteg
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    # Képek lapítása egyetlen vektorrá
    Flatten(),
    
    # Dropout réteg az overfitting csökkentésére
    Dropout(0.5),
    
    # Teljesen összekötött réteg
    Dense(512, activation='relu'),
    
    # Kimenet
    Dense(CLASSES, activation='softmax')  # Tíz osztály megkülönböztetéséhez
])

model.compile(loss='categorical_crossentropy',  # Többosztályos klasszifikációhoz
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [ ]:
# Modell tanítása
history = model.fit(
    train_generator,
    epochs=EPOCHS,  # Tanítási epochok száma
    validation_data=validation_generator,
    verbose=2)


In [ ]:
# A tanulási görbék ábrázolása

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'bo', label='Tanítási pontosság')
plt.plot(epochs, val_acc, 'b', label='Validációs pontosság')
plt.title('Tanítási és validációs pontosság')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo', label='Tanítási veszteség')
plt.plot(epochs, val_loss, 'b', label='Validációs veszteség')
plt.title('Tanítási és validációs veszteség')
plt.legend()

plt.show()